### Runnable demo is available at https://www.kaggle.com/joshuajulioadidjaja/soccer
### Please go to the link for direct running.

In [ ]:
library(tidyverse)
library(DBI)
library(RSQLite)
library(reshape2)
library(data.table)
library(dplyr)
library(knitr)
library(psych)
library(ggfortify)
library("factoextra")
library("FactoMineR")
library(yardstick)
library(ggplot2)
library(vcdExtra)

# I. Requirements

# II. Read data

In [ ]:
soccer <- dbConnect(SQLite(), 
                 "../input/soccer/database.sqlite")
## list all tables
tables <- dbListTables(soccer)

## exclude sqlite_sequence (contains table information)
tables <- tables[tables != "sqlite_sequence"]
lDataFrames <- vector("list", length=length(tables))

## create a data.frame for each table
for (i in seq(along=tables)) {
  lDataFrames[[i]] <- 
    dbGetQuery(conn=soccer, 
               statement=paste("SELECT * FROM '", 
                               tables[[i]], "'", sep=""))
}
## create a data.frame for each table
for (i in seq(along=tables)) {
  lDataFrames[[i]] <- 
    dbGetQuery(conn=soccer, 
               statement=paste("SELECT * FROM '", 
                               tables[[i]], "'", sep=""))
}

In [ ]:
# create dataframes
country <-  data.frame(lDataFrames[1])
league  <-  data.frame(lDataFrames[2])
match   <-  data.frame(lDataFrames[3])
player  <-  data.frame(lDataFrames[4])
team    <-  data.frame(lDataFrames[6])

# III. Preprocessing

### 1. Winning rate calculation

In [ ]:
scoring <- function(x){  
    if (x==0) {
        ans = 1/3
    }else if (x>0){
        ans = 1
    }else{
        ans = 0}
    return(ans)
    }
match_score <- match %>% select(match_api_id,date,home_team_api_id,away_team_api_id,home_team_goal, away_team_goal)
team <- team %>% select(team_api_id,team_long_name)
match_score$home_score <- (match$home_team_goal - match$away_team_goal)
match_score$away_score <- - match_score$home_score
match_score$home_win_rate <- as.numeric(lapply(match_score$home_score,FUN=scoring))
match_score$away_win_rate <- as.numeric(lapply(match_score$away_score,FUN=scoring))
match_home <- match_score %>% select(match_api_id,date,home_team_api_id,home_score,home_win_rate) %>% 
  rename(
    team_api_id = home_team_api_id,
    score = home_score,
    win_rate = home_win_rate
    )
match_away <- match_score %>% select(match_api_id,date,away_team_api_id,away_score, away_win_rate) %>% 
  rename(
    team_api_id = away_team_api_id,
    score = away_score,
    win_rate = away_win_rate
    )
total <- rbind(match_home, match_away)
total$date <- as.Date(total$date)
head(arrange(total,match_api_id))
match14 = with(total, total[(date >= "2014-01-01" & date < "2015-01-01"),])
match15 = with(total, total[(date >= "2015-01-01" & date < "2016-01-01"),])
df14 = subset(match14, select=-c(date))
df15 = subset(match15, select=-c(date))
team_ranking14_byscore <- aggregate(df14$score, by=list(team_api_id=df14$team_api_id), FUN=mean) %>% rename(score=x)
team_ranking15_byscore <- aggregate(df15$score, by=list(team_api_id=df15$team_api_id), FUN=mean) %>% rename(score=x)
team_ranking14_bywin <- aggregate(df14$win_rate, by=list(team_api_id=df14$team_api_id), FUN=mean) %>% rename(win_rate=x)
team_ranking15_bywin <- aggregate(df15$win_rate, by=list(team_api_id=df15$team_api_id), FUN=mean) %>% rename(win_rate=x)
team_ranking14 <- merge(team,team_ranking14_byscore, by="team_api_id",all.team_ranking14_byscore = TRUE)
team_ranking15 <- merge(team,team_ranking15_byscore,by="team_api_id",all.team_ranking14_byscore = TRUE)
team_ranking14 <- arrange(merge(team_ranking14,team_ranking14_bywin, by="team_api_id"),-win_rate)
team_ranking15 <- arrange(merge(team_ranking15,team_ranking15_bywin,by="team_api_id"),-win_rate)

In [ ]:
team_ranking14
team_ranking15

### 2. Team - players aggregation

In [ ]:
team = data.frame(lDataFrames[6])
# select relevant columns
country <- select(country, id, name) %>% rename(country_id = id)  %>% rename(country_name = name)   # use country_id as key for join
league  <- select(league, country_id, name) %>% rename(league_name = name) # use country_id as key for join
match   <- select(match, id, country_id, league_id, season, stage, date, match_api_id, home_team_api_id, away_team_api_id, home_team_goal, away_team_goal, home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, home_player_6, home_player_7, home_player_8, home_player_9, home_player_10, home_player_11, away_player_1, away_player_2, away_player_3, away_player_4, away_player_5, away_player_6, away_player_7, away_player_8, away_player_9, away_player_10, away_player_11, goal, shoton, shotoff, foulcommit, card, cross, corner, possession)
player  <- select(player,player_api_id, player_name) # use player_api_id as key for join
team    <- select(team, team_api_id, team_long_name, team_short_name) # use team_api_id as key for join

# agrregation
others = colnames(match)[1:11]
players = colnames(match)[12:33]
flatten <- melt(match, id = others, measure = players, na.rm = TRUE, value.name = "player_api_id") %>% 
  mutate(team_api_id = 
           ifelse(grepl("home",variable),home_team_api_id, 
                  ifelse(grepl("away",variable),away_team_api_id,NA))) %>%  
  left_join(country, by = "country_id")    %>% 
  left_join(league,  by = "country_id")    %>%  
  left_join(team,    by = "team_api_id")   %>%
  left_join(player,  by = "player_api_id") %>% 
  separate(season, into = c("season_start","season_end"), sep = "/", convert = TRUE) 
head(flatten)

# Filter
team_players <- select(flatten, season_start, season_end, 
                       country_name, league_name, player_api_id, 
                       team_api_id, team_long_name, team_short_name,
                       player_name)

team_players_2015 <- team_players %>% filter(season_start == 2015)
team_players_2014 <- team_players %>% filter(season_start == 2014)

In [ ]:
head(team_players_2014)
head(team_players_2015)

In [ ]:
head(team_ranking14)
head(team_ranking15)

### 3. Players

In [ ]:
player <- dbReadTable(soccer, 'Player')
player_attr <- dbReadTable(soccer, 'Player_Attributes')

player <- as.data.frame(player)
player_attr <- as.data.frame(player_attr)
player_attr$date <- substr(player_attr$date,1,10)

# filter season date
player_attr_2014 <- subset(player_attr, 
                           date >= '2014-07-18' & date <= '2015-05-31')
player_attr_2015 <- subset(player_attr, 
                           date >= '2015-07-17' & date <= '2016-05-25')

# remove negative values
player[player < 0] <- NA
player_attr_2014[player_attr_2014 < 0] <- NA
player_attr_2015[player_attr_2015 < 0] <- NA

# merge values of one player to one
aggregate_process <- function(x) {
  if (is.numeric(x)) {mean(x)}
  else {x[1]}
}

player_attr_2014_aggregate <- aggregate(player_attr_2014, 
               by=list(player_attr_2014$player_api_id),
               aggregate_process)
player_attr_2014 <- 
  player_attr_2014_aggregate[2:ncol(player_attr_2014_aggregate)]

player_attr_2015_aggregate <- aggregate(player_attr_2015, 
                                        by=list(player_attr_2015$player_api_id),
                                        aggregate_process)
player_attr_2015 <- 
  player_attr_2015_aggregate[2:ncol(player_attr_2015_aggregate)]

# remove outliers
# normalize
preprocess <- function(data) {
  for(i in 1:ncol(data)) {
    if (is.numeric(data[,i]) && !grepl("id",names(data)[i])) {
      mean <- mean(data[,i])
      sd <- sd(data[,i])
      data[,i] <- replace(data[,i],data[,i]>mean+2*sd,NA)
      data[,i] <- replace(data[,i],data[,i]<mean-2*sd,NA)
      data[,i] <- scale(data[,i])
    }
  }
  return(data)
}

player <- preprocess(player)
player_attr_2014 <- preprocess(player_attr_2014)
player_attr_2015 <- preprocess(player_attr_2015)
# merge with players' basic info
player_2014 <- merge(player,player_attr_2014[,c(2:42)],
                     by=c('player_api_id','player_fifa_api_id'),
                     all.x = TRUE, all.y = TRUE)
player_2015 <- merge(player,player_attr_2015[,c(2:42)],
                     by=c('player_api_id','player_fifa_api_id'),
                     all.x = TRUE, all.y = TRUE)
# change dummies
player_2014$left_foot <- ifelse(player_2014$preferred_foot == 'left', 1, 0)
player_2015$left_foot <- ifelse(player_2015$preferred_foot == 'left', 1, 0)

player_2014$attacking_work_rate <- replace(
  player_2014$attacking_work_rate,player_2014$attacking_work_rate=='None',NA)
player_2014$attack_high <- ifelse(player_2014$attacking_work_rate == 'high', 1, 0)
player_2014$attack_medium <- ifelse(player_2014$attacking_work_rate == 'medium', 1, 0)

player_2015$attacking_work_rate <- replace(
  player_2015$attacking_work_rate,player_2015$attacking_work_rate=='None',NA)
player_2015$attack_high <- ifelse(player_2015$attacking_work_rate == 'high', 1, 0)
player_2015$attack_medium <- ifelse(player_2015$attacking_work_rate == 'medium', 1, 0)

player_2014$defend_high <- ifelse(player_2014$defensive_work_rate == 'high', 1, 0)
player_2014$defend_medium <- ifelse(player_2014$defensive_work_rate == 'medium', 1, 0)
player_2015$defend_high <- ifelse(player_2015$defensive_work_rate == 'high', 1, 0)
player_2015$defend_medium <- ifelse(player_2015$defensive_work_rate == 'medium', 1, 0)

In [ ]:
head(player_2014)
head(player_2015)

### 4. Team


In [ ]:
team <- data.frame(lDataFrames[6])
team_attr <- data.frame(lDataFrames[7])

team <- select(team,id,team_api_id)
team_attr$date <- substr(team_attr$date,1,10)
team_attr <- select(team_attr,id,team_api_id,date,
                    buildUpPlaySpeed,buildUpPlayDribbling,
                    buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,
                    chanceCreationShooting,chanceCreationPositioningClass,defencePressure,
                    defenceAggression,defenceTeamWidth,defenceDefenderLineClass)

#filter season date
team_attr_2014 <- subset(team_attr, date >= '2014-07-18' & date <= '2015-05-31')
team_attr_2015 <- subset(team_attr, date >= '2015-07-17' & date <= '2016-05-25')

# remove negative values
team_attr_2014[team_attr_2014 < 0] <- NA
team_attr_2015[team_attr_2015 < 0] <- NA

# merge values of one team to one
aggregate_process <- function(x) {
  if (is.numeric(x)) {mean(x)}
  else {x[1]}
}

team_attr_2014_aggregate <- aggregate(team_attr_2014, 
                                      by=list(team_attr_2014$team_api_id),
                                      aggregate_process)
team_attr_2014 <- 
  team_attr_2014_aggregate[2:ncol(team_attr_2014_aggregate)]

team_attr_2015_aggregate <- aggregate(team_attr_2015, 
                                      by=list(team_attr_2015$team_api_id),
                                      aggregate_process)
team_attr_2015 <- 
  team_attr_2015_aggregate[2:ncol(team_attr_2015_aggregate)]

# remove outliers and normalize
preprocess <- function(data) {
  for(i in 1:ncol(data)) {
    if (is.numeric(data[,i]) && (grepl("Width",names(data)[i]) | !grepl("id",names(data)[i]))) {
      mean <- mean(data[,i])
      sd <- sd(data[,i])
      data[,i] <- replace(data[,i],data[,i]>mean+2*sd,NA)
      data[,i] <- replace(data[,i],data[,i]<mean-2*sd,NA)
      data[,i] <- scale(data[,i])
    }
  }
  return(data)
}

team_attr_2014 <- preprocess(team_attr_2014)
team_attr_2015 <- preprocess(team_attr_2015)

# merge with team's rank
team_2014 <- merge(team_ranking14,team_attr_2014[,c(2:14)],
                   by=c('team_api_id'),
                   all.x = TRUE, all.y = TRUE)
team_2015 <- merge(team_ranking15,team_attr_2015[,c(2:14)],
                   by=c('team_api_id'),
                   all.x = TRUE, all.y = TRUE)

team_2014[is.na(team_2014)] <- 0
team_2015[is.na(team_2015)] <- 0

In [ ]:
team_ranking15

In [ ]:
head(team_2014)
head(team_2015)

# IV. Comparison between leagues

In [ ]:
full_league_stat_14 <- team_players_2014 %>% left_join(player_2014, by = 'player_api_id') %>%
    select(c('league_name', 'overall_rating')) %>%
    group_by_at(1) %>%
    summarise_each(funs(mean(., na.rm=T))) %>% 
    left_join(league, by = 'league_name') %>% 
    left_join(data.frame(lDataFrames[1]), by = c('country_id'='id'))
full_league_stat_14[2,4] = 'UK'
full_league_stat_14
library(ggplot2)
require(maps)

# map of europe
some.eu.countries <- full_league_stat_14$name
eur <- map_data("world", region = some.eu.countries) %>% left_join(full_league_stat_14, by = c('region' = 'name'))
region.label <- eur %>% 
  group_by(region) %>%
  summarise_each(funs(mean), c(1:2))


ggplot(eur, aes(x = long, y = lat)) +
  geom_polygon(aes(group = group, fill = overall_rating))+
  geom_text(aes(label = region), data = region.label,  size = 3, hjust = 0.5)+
  coord_fixed(1.3) +
  theme(legend.position = "right")

# V. Spearman correlation test

In [ ]:
player_att  <-  data.frame(lDataFrames[5])
player_att  <- select(player_att,player_api_id, date, overall_rating, potential)
player_att14 = select(with(player_att, player_att[(date >= "2014-07-18" & date <= "2015-05-31"),]),-c(date))
player_att15 = select(with(player_att, player_att[(date >= "2015-07-17" & date <= "2016-05-25"),]), -c(date))

# Average potential and overall_rating by player over a season
player14 <- player_att14 %>% group_by(player_api_id) %>% summarise_all("mean")
player15 <- player_att15 %>% group_by(player_api_id) %>% summarise_all("mean")

# Match players with teams
team14 <- select(team_players_2014,player_api_id,team_api_id)
team14 <- merge(team14,player14,by="player_api_id") %>% distinct()

team15 <- select(team_players_2015,player_api_id,team_api_id)
team15 <- merge(team15,player15,by="player_api_id") %>% distinct()

# find average player for each team for that season
player_ranking14 <- select(team14,-c(player_api_id)) %>% group_by(team_api_id) %>% summarise_all("mean")
player_ranking15 <- select(team15,-c(player_api_id)) %>% group_by(team_api_id) %>% summarise_all("mean")


# Create dataframe with team winning-rate and player's abilities
final14 <- arrange(merge(team_ranking14,player_ranking14, by="team_api_id"),-win_rate)
final15 <- arrange(merge(team_ranking15,player_ranking15, by="team_api_id"),-win_rate)

In [ ]:
library(ggplot2)
ggplot(final14, aes(x=win_rate, y=overall_rating)) + 
  geom_point(color='#2980B9', size = 4) + 
  geom_smooth(method=lm, se=FALSE, fullrange=TRUE, color='#2C3E50')
ggplot(final14, aes(x=win_rate, y=potential)) + 
  geom_point(color='#2980B9', size = 4) + 
  geom_smooth(method=lm, se=FALSE, fullrange=TRUE, color='#2C3E50')


ggplot(final15, aes(x=win_rate, y=overall_rating)) + 
  geom_point(color='#2980B9', size = 4) + 
  geom_smooth(method=lm, se=FALSE, fullrange=TRUE, color='#2C3E50')
ggplot(final15, aes(x=win_rate, y=potential)) + 
  geom_point(color='#2980B9', size = 4) + 
  geom_smooth(method=lm, se=FALSE, fullrange=TRUE, color='#2C3E50')

In [ ]:
# Correlate by Overall-rating and Winning-rate
corr14 <- cor.test(x=final14$win_rate, y=final14$overall_rating, method = 'spearman',conf.level = 0.95)
corr15 <- cor.test(x=final15$win_rate, y=final14$overall_rating, method = 'spearman',conf.level = 0.95)

corr14
corr15

# V. Player feature analysis

### 1. Filter interested leagues

In [ ]:
# filter only 3 leagues
player_name_2014 <- subset(team_players_2014$player_name,
  team_players_2014$league_name == "England Premier League" |
  team_players_2014$league_name == "Germany 1. Bundesliga" |
  team_players_2014$league_name == "Spain LIGA BBVA"
)
player_2014 <- subset(player_2014,
                      player_name %in% player_name_2014)
player_name_2015 <- subset(team_players_2015$player_name,
  team_players_2015$league_name == "England Premier League" |
  team_players_2015$league_name == "Germany 1. Bundesliga" |
  team_players_2015$league_name == "Spain LIGA BBVA"
)
player_2015 <- subset(player_2015,
                      player_name %in% player_name_2015)

## filter for EACH league
player_name_2014_eng <- subset(team_players_2014$player_name,
  team_players_2014$league_name == "England Premier League")
player_name_2014_ger <- subset(team_players_2014$player_name,
  team_players_2014$league_name == "Germany 1. Bundesliga")
player_name_2014_spa <- subset(team_players_2014$player_name,
  team_players_2014$league_name == "Spain LIGA BBVA")

player_2014_eng <- subset(player_2014,
                      player_name %in% player_name_2014_eng)

player_2014_ger <- subset(player_2014,
                          player_name %in% player_name_2014_ger)

player_2014_spa <- subset(player_2014,
                          player_name %in% player_name_2014_spa)

### 2. Linear regression

In [ ]:
player_2014.lr <-
  lm(overall_rating ~ 
       height+weight+crossing+finishing+heading_accuracy+short_passing+
     volleys+dribbling+curve+free_kick_accuracy+long_passing+ball_control+
     acceleration+sprint_speed+agility+reactions+balance+shot_power+jumping+
     stamina+strength+long_shots+aggression+interceptions+positioning+vision+
     penalties+marking+standing_tackle+sliding_tackle+gk_diving+gk_handling+
     gk_kicking+gk_positioning+gk_reflexes+left_foot+attack_high+
     attack_medium+defend_high+defend_medium,
   na.action = na.exclude, data = player_2014)
summary(player_2014.lr)

player_2014_eng.lr <-
  lm(overall_rating ~ 
       height+weight+crossing+finishing+heading_accuracy+short_passing+
       volleys+dribbling+curve+free_kick_accuracy+long_passing+ball_control+
       acceleration+sprint_speed+agility+reactions+balance+shot_power+jumping+
       stamina+strength+long_shots+aggression+interceptions+positioning+vision+
       penalties+marking+standing_tackle+sliding_tackle+gk_diving+gk_handling+
       gk_kicking+gk_positioning+gk_reflexes+left_foot+attack_high+
       attack_medium+defend_high+defend_medium,
     na.action = na.exclude, data = player_2014_eng)
summary(player_2014_eng.lr)

player_2014_ger.lr <-
  lm(overall_rating ~ 
       height+weight+crossing+finishing+heading_accuracy+short_passing+
       volleys+dribbling+curve+free_kick_accuracy+long_passing+ball_control+
       acceleration+sprint_speed+agility+reactions+balance+shot_power+jumping+
       stamina+strength+long_shots+aggression+interceptions+positioning+vision+
       penalties+marking+standing_tackle+sliding_tackle+gk_diving+gk_handling+
       gk_kicking+gk_positioning+gk_reflexes+left_foot+attack_high+
       attack_medium+defend_high+defend_medium,
     na.action = na.exclude, data = player_2014_ger)
summary(player_2014_ger.lr)

player_2014_spa.lr <-
  lm(overall_rating ~ 
       height+weight+crossing+finishing+heading_accuracy+short_passing+
       volleys+dribbling+curve+free_kick_accuracy+long_passing+ball_control+
       acceleration+sprint_speed+agility+reactions+balance+shot_power+jumping+
       stamina+strength+long_shots+aggression+interceptions+positioning+vision+
       penalties+marking+standing_tackle+sliding_tackle+gk_diving+gk_handling+
       gk_kicking+gk_positioning+gk_reflexes+left_foot+attack_high+
       attack_medium+defend_high+defend_medium,
     na.action = na.exclude, data = player_2014_spa)
summary(player_2014_spa.lr)

library(ggplot2)
ggplot(data=player_2014_eng, 
       aes(x=player_2014_eng$aggression, y=player_2014_eng$overall_rating, 
           col=player_2014_eng$height, size=player_2014_eng$stamina)) + 
  geom_point()

player_2014.res <- resid(player_2014.lr)
plot(player_2014$overall_rating, player_2014.res,
     ylab="Residuals", xlab="Overall Rating")
abline(0,0)

plot(player_2014.lr)

player_2015.predict <- predict(player_2014.lr, player_2015[,c(6,7,14:51)],
        level=0.95)

library(forecast)
accuracy(player_2015.predict, player_2015$overall_rating)

# VI. Team features analysis

### 1. Filter interested leagues

In [ ]:
team_id_2014 <- team_players_2014$team_api_id
team_2014 <- subset(team_2014,
                    team_api_id %in% team_id_2014)

team_id_2014_eng <- subset(team_players_2014$team_api_id,
                           team_players_2014$league_name == "England Premier League")
team_2014_eng <- subset(team_2014,
                        team_api_id %in% team_id_2014_eng)

team_id_2014_ger <- subset(team_players_2014$team_api_id,
                           team_players_2014$league_name == "Germany 1. Bundesliga")
team_2014_ger <- subset(team_2014,
                        team_api_id %in% team_id_2014_ger)

team_id_2014_spa <- subset(team_players_2014$team_api_id,
                           team_players_2014$league_name == "Spain LIGA BBVA")
team_2014_spa <- subset(team_2014,
                        team_api_id %in% team_id_2014_spa)

team_id_2015 <- team_players_2015$team_api_id
team_2015 <- subset(team_2015,
                    team_api_id %in% team_id_2015)

team_id_2015_eng <- subset(team_players_2015$team_api_id,
                           team_players_2015$league_name == "England Premier League")
team_2015_eng <- subset(team_2015,
                        team_api_id %in% team_id_2015_eng)

team_id_2015_ger <- subset(team_players_2015$team_api_id,
                           team_players_2015$league_name == "Germany 1. Bundesliga")
team_2015_ger <- subset(team_2015,
                        team_api_id %in% team_id_2015_ger)

team_id_2015_spa <- subset(team_players_2015$team_api_id,
                           team_players_2015$league_name == "Spain LIGA BBVA")
team_2015_spa <- subset(team_2015,
                        team_api_id %in% team_id_2015_spa)


# change dummies
team_2014$chanceCreationPositioning_organised <- ifelse(team_2014$chanceCreationPositioningClass == "Organised", 1, 0)
team_2015$chanceCreationPositioning_organised <- ifelse(team_2015$chanceCreationPositioningClass == "Organised", 1, 0)

team_2014$defenceDefenderLine_cover <- ifelse(team_2014$defenceDefenderLineClass == "Cover", 1, 0)
team_2015$defenceDefenderLine_cover <- ifelse(team_2015$defenceDefenderLineClass == "Cover", 1, 0)

### 2. PCA

In [ ]:
## England
team_2014_eng_num <- team_2014_eng[,c(6:11,13:15)]
eng.pca <- PCA(team_2014_eng_num, graph = FALSE)
fviz_pca_var(eng.pca, col.var = "contrib",
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"), 
             repel = TRUE) # Avoid text overlapping)

## Germany
             
team_2014_ger_num <- team_2014_ger[,c(6:11,13:15)]
ger.pca <- PCA(team_2014_ger_num, graph = FALSE)
fviz_pca_var(ger.pca, col.var = "contrib",
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"), 
             repel = TRUE) # Avoid text overlapping

## Spain
team_2014_spa_num <- team_2014_spa[,c(6:11,13:15)]
spa.pca <- PCA(team_2014_spa_num, graph = FALSE)
fviz_pca_var(spa.pca, col.var = "contrib",
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"), 
             repel = TRUE) # Avoid text overlapping

### 3. Linear regression

In [ ]:
### linear regression
library(forecast)

## England
team_2014_eng_win_rate.lr <- lm(win_rate ~ buildUpPlaySpeed+buildUpPlayDribbling+buildUpPlayPassing+
                                  chanceCreationPassing+chanceCreationCrossing+chanceCreationShooting+
                                  defencePressure+defenceAggression+defenceTeamWidth,
                                na.action = na.exclude, data = team_2014_eng)
summary(team_2014_eng_win_rate.lr)


## Germany
team_2014_ger_win_rate.lr <- lm(win_rate ~ buildUpPlaySpeed+buildUpPlayDribbling+buildUpPlayPassing+
                                  chanceCreationCrossing+chanceCreationShooting+
                                  defencePressure+defenceTeamWidth,
                                na.action = na.exclude, data = team_2014_ger)
summary(team_2014_ger_win_rate.lr)


## Spain
team_2014_spa_win_rate.lr <- lm(win_rate ~ buildUpPlaySpeed+buildUpPlayDribbling+buildUpPlayPassing+
                                  chanceCreationPassing+chanceCreationCrossing+chanceCreationShooting+
                                  defencePressure+defenceAggression+defenceTeamWidth,
                                na.action = na.exclude, data = team_2014_spa)
summary(team_2014_spa_win_rate.lr)


# VII. Win prediction

### 1. Gather independent variables

In [ ]:
player_2014$left_foot <- ifelse(player_2014$preferred_foot == 'left', 1, 0)
player_2014$right_foot <- ifelse(player_2014$preferred_foot == 'right', 1, 0)
player_2015$left_foot <- ifelse(player_2015$preferred_foot == 'left', 1, 0)
player_2015$right_foot <- ifelse(player_2015$preferred_foot == 'right', 1, 0)
full_team_stat_14 <- team_players_2014 %>% left_join(player_2014, by = 'player_api_id') %>%
    select(-c('player_name.y', 'player_name.x', 'birthday', 'preferred_foot', 'date', 'id')) %>% 
    group_by_at(c(1:4, 6:8)) %>%
    summarise_each(funs(ave(.,na.rm=T)), c(9:47)) %>% 
    distinct() 
full_team_stat_14 <- left_join(full_team_stat_14, team_attr_2014, by = 'team_api_id') %>%
    select(-c('chanceCreationPositioningClass', 'defenceDefenderLineClass'))
full_team_stat_15 <- team_players_2015 %>% left_join(player_2015, by = 'player_api_id') %>%
    select(-c('player_name.y', 'player_name.x', 'birthday', 'preferred_foot', 'date', 'id')) %>% 
    group_by_at(c(1:4, 6:8)) %>%
    summarise_each(funs(ave(.,na.rm=T)), c(9:47)) %>% 
    distinct() 
full_team_stat_15 <- left_join(full_team_stat_15, team_attr_2015, by = 'team_api_id') %>%
    select(-c('chanceCreationPositioningClass', 'defenceDefenderLineClass'))
country = 'Spain'
full_team_stat_14_league <- full_team_stat_14 %>% 
    filter(season_start == 2014 & country_name == country) %>%
    ungroup() %>%
    select(-c('season_start', 'season_end', 'country_name', 'league_name', 'id'))
full_team_stat_15_league <- full_team_stat_15 %>% 
    filter(season_start == 2015 & country_name == country) %>%
    ungroup() %>%
    select(-c('season_start', 'season_end', 'country_name', 'league_name', 'id'))

In [ ]:
head(full_team_stat_14_league)
head(full_team_stat_15_league)

### 2. Tabulate the matches

In [ ]:
match_14 <- match %>% filter(season == '2014/2015')
match_stat_14 <- match_14 %>% right_join(full_team_stat_14_league, by = c('home_team_api_id'='team_api_id')) %>%
                right_join(full_team_stat_14_league, by = c('away_team_api_id'='team_api_id')) %>%
                select_at(c(7:11, 44:92, 95:143)) %>%
                select(-c('date', 'date.y'))
match_15 <- match %>% filter(season == '2014/2015')
match_stat_15 <- match_15 %>% right_join(full_team_stat_15_league, by = c('home_team_api_id'='team_api_id')) %>%
                right_join(full_team_stat_15_league, by = c('away_team_api_id'='team_api_id')) %>%
                select_at(c(7:11, 44:92, 95:143)) %>%
                select(-c('date', 'date.y')) %>%
                drop_na()
match_stat_14$result = ifelse(match_stat_14$away_team_goal < match_stat_14$home_team_goal, 1, 0)
result = ifelse(match_stat_14$away_team_goal < match_stat_14$home_team_goal, 1, 0)
match_stat_15$result = ifelse(match_stat_15$away_team_goal < match_stat_15$home_team_goal, 1, 0)

### 3. Logistic regression

In [ ]:
cor(match_stat_14, use = "complete.obs")

In [ ]:
# First pass
d <- cbind(result, match_stat_14[6:98])
lmf <- reformulate(names(d)[-1], names(d[1]))
model.fit <- glm(lmf, data = match_stat_14, family="binomial", na.action = na.omit)

# Eliminate correlated features
nn <- names(na.exclude(model.fit$coefficients))[-1]
lmf <- reformulate(nn, 'result')
model.fit <- glm(lmf, data = match_stat_14, family="binomial", na.action = na.omit)

### 4. Prediction

In [ ]:
# Prediction
model.prob = predict(model.fit, match_stat_15, type="response")
model.pred = rep(0, dim(match_stat_15)[1])
model.pred[model.prob > .5] = 1
a<- table(model.pred, match_stat_15$result)
mean(model.pred == match_stat_15$result)
# The confusion matrix from a single assessment set (i.e. fold)
cm <- conf_mat(a, match_stat_15$result, model.pred)
autoplot(cm, type = "heatmap") +
  scale_fill_gradient(low="#D6EAF8",high = "#2E86C1") +
  theme(text = element_text(size=20))
HLtest(model = model.fit)

### 5. League competitiveness


In [ ]:
data <- left_join(team_ranking15, full_team_stat_15, by = 'team_api_id')
eng <- data %>% filter(country_name %in% c('England', 'Germany', 'Spain'))
boxplot(score ~ country_name, data = eng,
        varwidth = TRUE, las = 1, col = 3:5)

In [ ]:
data <- team_players_2014 %>% left_join(player_2014, by = 'player_api_id') %>%
    select(c('team_api_id', 'overall_rating')) %>%
    group_by_at(1) %>%
    summarise_each(funs(mean(., na.rm=T))) %>%
    left_join(full_team_stat_15, by = 'team_api_id')
eng <- data %>% filter(country_name %in% c('England', 'Germany', 'Spain'))
boxplot(overall_rating ~ country_name, data = eng,
        varwidth = TRUE, las = 1, col = 3:5)

# VIII. Home advantage

In [ ]:
country <-  data.frame(lDataFrames[1])
country <- select(country, id, name) %>% rename(country_id = id)  %>% rename(country_name = name)   # use country_id as key for join
match <- data.frame(lDataFrames[3])

match <- match %>% select(match_api_id,country_id, season,home_team_api_id,home_team_goal, away_team_goal)
match$home_score <- (match$home_team_goal - match$away_team_goal)
match$away_score <- (match$away_team_goal - match$home_team_goal)
match <- left_join(match, country, by = "country_id")

match$home_win_rate <- as.numeric(lapply(match$home_score,FUN=scoring))
match$away_win_rate <- as.numeric(lapply(match$away_score,FUN=scoring))
match <- match %>% select(season,country_name,home_win_rate,away_win_rate)

match15 = with(match, match[(season=='2015/2016'),])
match15 <- match15 %>% 
            group_by(country_name) %>% 
               summarise_at(vars("home_win_rate", "away_win_rate"), mean)
match15$home_adv <-  (match15$home_win_rate - match15$away_win_rate)*100
match15[2,1] = 'UK'
match15
match8 = with(match, match[(season=='2008/2009'),])
match8 <- match8 %>% 
            group_by(country_name) %>% 
               summarise_at(vars("home_win_rate", "away_win_rate"), mean)
match8$home_adv <-  (match8$home_win_rate - match8$away_win_rate)*100
match8[2,1] = 'UK'
match8

In [ ]:
require(maps)

# map of europe
some.eu.countries <- match8$country_name
eur <- map_data("world", region = some.eu.countries) %>% left_join(match8, by = c('region' = 'country_name'))
region.label <- eur %>% 
  group_by(region) %>%
  summarise_each(funs(mean), c(1:2))

ggplot(eur, aes(x = long, y = lat)) +
  geom_polygon(aes(group = group, fill = home_adv))+
  geom_text(aes(label = region), data = region.label,  size = 4, hjust = 0.5, color="black")+
  coord_fixed(1.3) +
  scale_fill_viridis_c(option = "C",direction = -1,alpha = 1,begin = 0.1,end = 1, limits=c(5, 27),guide_legend(title="Home advantage in %"))+
  ggtitle("Home advantage in 2008/2009 season") +
  theme_void()+
  theme(legend.position = "right",plot.title = element_text(hjust = 0.5))